In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Numeric, DateTime
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.automap import automap_base

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:

engine = create_engine(connection_string, echo=False)
meta = MetaData()

nome_tabela = 'COVID'

covid = Table('covid', meta, 
    Column("DATE", Date),
    Column("TYPE", String(30)),
    Column("CASES", Integer),
    Column("DIFFERENCE", Integer),
    Column("COUNTRY", String(50)),
    Column("PROVINCE_STATE", String(50)), 
    Column("LAT", Numeric(18,8)),
    Column("LONG", Numeric(18,8)),
    Column("LATEST_DATE", DateTime))

if not engine.dialect.has_table(engine, nome_tabela, schema = 'BLUADMIN'):
    meta.create_all(engine)
else:
    conn = engine.connect()
    #stmt = covid.delete().where(students.c.lastname == 'Khanna')
    stmt = covid.delete()
    conn.execute(stmt)
    conn.close()

In [4]:
Base = automap_base()

#engine = create_engine(connection_string, echo=True)
session = scoped_session(sessionmaker(autocommit=True,
                                      autoflush=False,
                                      bind=engine))

Base.prepare(engine, reflect=True)
base_sa = Base.classes

In [5]:
df = pd.read_csv('https://download.data.world/s/3xpcwdh7es3o6uqyztkkhdnh4ht7i7')
df.head()

,DATE,TYPE,CASES,DIFFERENCE,COUNTRY,PROVINCE_STATE,LAT,LONG,LATEST_DATE
0,2020-05-20,Deaths,0,0,France,French Polynesia,-17.6797,149.4068,2020-05-20T23:29:52
1,2020-05-20,Deaths,15,0,Congo (Brazzaville),NaN,-4.2634,15.2832,2020-05-20T23:29:52
2,2020-05-20,Confirmed,42,0,Burundi,NaN,-3.3731,29.9189,2020-05-20T23:29:52
3,2020-05-20,Deaths,9,0,China,Beijing,40.1824,116.4142,2020-05-20T23:29:52
4,2020-05-20,Confirmed,2467,21,Canada,British Columbia,53.7267,-127.6476,2020-05-20T23:29:52


In [6]:
nome_tabela = 'covid'

In [7]:
df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False)
#df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False, chunksize=1000)   # With number of records to be processed each time

In [8]:
df2 = pd.read_sql_table(nome_tabela, engine)
df2.head()

,DATE,TYPE,cases,difference,country,province_state,lat,LONG,latest_date
0,2020-05-20,Deaths,4222,23,Turkey,None,38.963700,35.243300,2020-05-20 23:29:52
1,2020-05-20,Confirmed,746,8,Uruguay,None,-32.522800,-55.765800,2020-05-20 23:29:52
2,2020-05-20,Confirmed,2728,0,Sudan,None,12.862800,30.217600,2020-05-20 23:29:52
3,2020-05-20,Deaths,56,0,Thailand,None,15.870032,100.992541,2020-05-20 23:29:52
4,2020-05-20,Confirmed,71,1,Malawi,None,-13.254300,34.301500,2020-05-20 23:29:52


In [9]:
session.close()